# Honey Supply Policy 

The current Honey supply policy depends on governance to adjust a per block issuance rate. This is not ideal because it requires administrative governance to modulate the issuance rate, making it difficult for participants who do not have a controlling interest in governance to have confidence that issuance will be handled responsibly and predictably over time. 

We would like to design and implement a dynamic issuance policy that minimizes the need for governance decisions over issuance while ensuring that the system is attractive to both more passive speculators that want to understand the upper bounds of issuance as well as active contributors who want to ensure that the system can provide adequate and consistent rewards for the value they produce.  
    
For the purpose of this model we differentiate the honey supply into two buckets, the common pool and the circulating supply. We model inflows to the system as parameter to show how the system would behave when inflows exceed outflows, and we treat conviction outflows as a fixed percent rate from the common pool over time. This abstraction of conviction outflows represents a **maximum possible rate for a given parameterization of conviction voting**, though in practice the real outflow rate would tend to be much lower due to the ability for participants to support signaling proposals (eg abstain), or for proposal to recieve some support but ultimately fail to reach sufficient support to execute. Never the less, this creates an outlfow boundary, and by modulating issuance based on inflows and outflows, we can also understand it as an issuance boundary. 

## State Variables

We define three state variables, the common pool balance, the circulating supply. All of these variables are denominated in honey.  


In [1]:
import random as rand

In [2]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# List of all the state variables in the system and their initial values
genesis_states = {
    'reserve': 7473, # Current common pool balance
    'circulation': 18765, # Total honey supply minus common pool balance (26238 - 7473)
    'netflow': 0.0,
    'adjustment':0.0
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Model Params
We define some parameters which can be used to tune the behavior of issuance and distribution in the model. Paramters assume that each time step of the model relfect 1 month of real time, the simulation will run for 120 timesteps giving the model as a whole a 10 year time horizon.  

In [3]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# List of all the state variables in the system and their initial values
params = {
    'outflow_rate': [0.05, 0.05, 0.025, 0.05], # proportion of common pool funds that leave each timestep (month), 
    'inflow': [0.0, 0.0, 0.0, 0.02], # proportion of circulating supply that is moved to the common pool each timestep, used to show how burning kicks in when inflows exceed outflows while. 
    'throttle': [0.0, 0.004, 0.0, 0.0], # maximimum proportion of the total supply that can be issued or burned in each timestep (month) 
    'target_reserve_ratio': [0.20, 0.20, 0.20, 0.20] # target ratio of common pool funds to total supply
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Timestep
We assume that each timestep in cadcad represents 1 month in real time, scaling model parameters to reflect that timescale. 

# Policies and State Update Functions

We define a supply policy and four state update fuctions. 


In [4]:
def supply_policy(params, step, sH, s):
    # first we calculate net flows
    outflow_rate = params['outflow_rate']
    # outflow = s['reserve'] * rand.triangular(0, outflow_rate, outflow_rate/5)
    outflow = s['reserve'] * outflow_rate
    netflow = params['inflow'] * s['circulation'] - outflow

    # then we calculate supply adjustments 
    reserve = s['reserve'] + netflow 
    circulation = s['circulation'] - netflow
    supply = reserve + circulation
    ratio = reserve / supply
    

    # Proportional control https://en.wikipedia.org/wiki/Proportional_control
    # corrections are made proportionally to the difference between the target and the current value

    e = (params['target_reserve_ratio'] - ratio) / 12

    if params['throttle'] != 0: 
        # Corrections bounded by a maximum issuance rate parameter 
        if e < 0:
            adjustment = max(e, -params['throttle']) * supply
        else:
            adjustment = min(e, params['throttle']) * supply 
    else:
        # Corrections are unbounded, issuance is bounded by the maximum outflow rate 
        adjustment = e * supply 


    return ({'netflow':netflow, 'adjustment':adjustment})


def update_reserve(params, step, sH, s, _input):
    key = 'reserve'
    value = s['reserve'] + _input['netflow'] + _input['adjustment']
    return (key, value)

def update_circulation(params, step, sH, s, _input):
    key = 'circulation'
    value = s['circulation'] - _input['netflow']
    return (key, value)

def update_netflow(params, step, sH, s, _input):
    key = 'netflow'
    value =  _input['netflow']
    return (key, value)

def update_adjustment(params, step, sH, s, _input):
    key = 'adjustment'
    value =  _input['adjustment']
    return (key, value)

# Partial State Update Blocks


In [5]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# In the Partial State Update Blocks, the user specifies if state update functions will be run in series or in parallel
partial_state_update_blocks = [
    { 
        'policies': {
            'driver': supply_policy
        },
        'variables': { # The following state variables will be updated simultaneously
            'reserve': update_reserve,
            'circulation': update_circulation,
            'netflow': update_netflow,
            'adjustment': update_adjustment
        }
    }
]
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Simulation Configuration Parameters
Lastly, we define the number of timesteps and the number of Monte Carlo runs of the simulation. These parameters must be passed in a dictionary, in `dict_keys` `T` and `N`, respectively. In our example, we'll run the simulation for 10 timesteps. And because we are dealing with a deterministic system, it makes no sense to have multiple Monte Carlo runs, so we set `N=1`. We'll ignore the `M` key for now and set it to an empty `dict`

In [6]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
# In this example, we'll run the simulation once (N=1) and its duration will be of 10 timesteps
# We'll cover the `M` key in a future article. For now, let's omit it
sim_config_dict = {
    'T': range(120),
    'N': 1,
    'M': params
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Putting it all together
We have defined the state variables of our system and their initial conditions, as well as the state update functions, which have been grouped in a single state update block. We have also specified the parameters of the simulation (number of timesteps and runs). We are now ready to put all those pieces together in a `Configuration` object.

In [7]:
#imported some addition utilities to help with configuration set-up
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs

exp = Experiment()
c = config_sim(sim_config_dict)
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
del configs[:]
exp.append_configs(initial_state=genesis_states, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_configs=c #preprocessed dictionaries containing simulation parameters
                      )

# Running the engine
We are now ready to run the engine with the configuration defined above. Instantiate an ExecutionMode, an ExecutionContext and an Executor objects, passing the Configuration object to the latter. Then run the `execute()` method of the Executor object, which returns the results of the experiment in the first element of a tuple.

In [8]:
%%capture
from cadCAD.engine import ExecutionMode, ExecutionContext
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(exec_mode.local_mode)

from cadCAD.engine import Executor

simulation = Executor(exec_context=local_mode_ctx, configs=configs) # Pass the configuration object inside an array
raw_system_events, tensor_field, sessions = simulation.execute() # The `execute()` method returns a tuple; its first elements contains the raw results


# Analyzing the results
We can now convert the raw results into a DataFrame for analysis

In [9]:
%matplotlib inline
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

reserve   circulation     netflow  \
subset run timestep substep                                          
0      1   0        0        7473.000000  18765.000000    0.000000   
           1        1        6945.037500  19138.650000 -373.650000   
           2        1        6482.698281  19485.901875 -347.251875   
           3        1        6078.159756  19810.036789 -324.134914   
           4        1        5724.534063  20113.944777 -303.907988   
...                                  ...           ...         ...   
3      3   116      1        3965.849142  12306.862024   48.031512   
           117      1        3950.431285  12259.017241   47.844783   
           118      1        3935.073369  12211.358460   47.658781   
           119      1        3919.775161  12163.884959   47.473501   
           120      1        3904.536429  12116.596018   47.288941   

                             adjustment  simulation  
subset run timestep substep                          
0      1   0        0          0.000000           0  
           1        1       -154.312500           0  
           2        1       -115.087344           0  
           3        1        -80.403611           0  
           4        1        -49.717705           0  
...                                 ...         ...  
3      3   116      1        -63.509545           0  
           117      1        -63.262641           0  
           118      1        -63.016697           0  
           119      1        -62.771709           0  
           120      1        -62.527673           0  

[484 rows x 5 columns]

# Analysis 

We have 4 different subsets of data based on different parameters. 

* subset 0: 20% reserve target, 0.05 outlfow rate from common pool per month, no throttle, no inflows
* subset 1: 20% reserve target, 0.05 outflow rate from common pool per month, 0.004 per month throttle, no inflows
* subset 2: 20% reserve target, 0.025 outflow rate from common pool per month, no throttle, no inflows
* subset 3: 20% reserve target, 0.05 outflow rate from the common pool per month, no throttle, and 0.02 inflows

I used a 20% reserve target and 0.05 maxiumum outflow rate as a baseline. Even with a 20% reserve target, the outflow rate and lack of inflows result in a equilibrium reserve of around 12%, if we conservatively assume the maximum outflow rate of 0.05 of the common pool, we end up with a max of 7.2% issuance per year. 

In the base case we do not specify the issuance rate, we only specify the target and conviction voting parameters that control the maximum outflow. However, if we want to define a maximum issuance rate directly we can use the throttle paramter. In subset 1, this is set to 0.004 or 4.8% per year, resulting in a reserve that stabilizes around 7.5%. It's worth noting that separting the max issuance rate from the conviction parameters, we can potentially remove governance over the max issuance (throttle), while retaining the ability to adjust conviction parameterization as needed, and we can more easily explain the maximum issuance rate without having to explain conviction voting parameterization in-depth. 

In practice we would expect the actual outflows to be much lower than theoretical maximum allows by a given set of conviction parameters. In subset 2, I turned the throttle back off but reduced the outflow rate by half. Reducing outflow rate results in slower overall supply growth, and the reserve stabilizing at a bit higher. From a system perspective, this means that if Honey holders choose to stake honey on the abstain proposals they would effectively be voting to reduce both the spending and the issuance rate. This ensures that honey holders do not have an incentive to overspend, because they always have the option to abstain and reduce issuance and outflows if they do not think any of the current proposals provide positive expected future value. 

Finally, in subset 4, I've set inflows to be greater than outflows, showing how over time this results in the reserves increasing beyond the target, and the supply of Honey decreasing as honey is burned from the common pool. Inflows are specified as a fixed ratio of the circulating supply in the model, which is completely arbitrary, but does help illustrate how consistent net inflows would impact both the total supply and reserves. Total supply would become deflationary, while the reserve would stabilize above the target reserve ratio. 






In [12]:
simulation_result['total_supply'] = simulation_result['reserve'] + simulation_result['circulation']
simulation_result['ratio'] = simulation_result['reserve'] / simulation_result['total_supply']
import plotly.express as px
fig = px.line(
    simulation_result,
    x='timestep',
    y=['total_supply', 'circulation', 'reserve'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [13]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['netflow', 'adjustment'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [15]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['ratio'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()